In [ ]:
import csv
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from os import getcwd
import matplotlib.pyplot as plt

In [ ]:
def get_data(filename):
    images = np.loadtxt(filename, delimiter=',',skiprows=1)
    labels = images[:, 0]
    images = images[:, 1:].astype('float64')
    images = np.reshape(images, (images.shape[0], 28, 28)) # turn (27455, 785) into (27455, 28, 28)
    return images, labels

In [ ]:
path_sign_mnist_train = f"{getcwd()}/data/sign_mnist_train.csv"
path_sign_mnist_test = f"{getcwd()}/data/sign_mnist_test.csv"
training_images, training_labels = get_data(path_sign_mnist_train)
testing_images, testing_labels = get_data(path_sign_mnist_test)

print(training_images.shape)
print(training_labels.shape)
print(testing_images.shape)
print(testing_labels.shape)

In [ ]:
training_images = np.expand_dims(training_images, axis=3) # turn (10000, 28, 28) into (10000, 28, 28, 1)
testing_images = np.expand_dims(testing_images, axis=3)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(
    rescale = 1./255
)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(25, activation='softmax')  # 25 outputs, softmax assigns a probability to each one
])

In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_datagen.flow(training_images,training_labels, batch_size=32),
    epochs=2,
    validation_data=validation_datagen.flow(testing_images,testing_labels, batch_size=32),
    verbose=2
)

In [ ]:
test = model.evaluate(testing_images, testing_labels, verbose=1)

In [ ]:
%matplotlib inline

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()